In [1]:
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.async
import kotlinx.coroutines.awaitAll
import kotlinx.coroutines.runBlocking
import com.google.common.hash.Hashing

In [2]:
import io.github.cdimascio.dotenv.Dotenv
import java.nio.file.Paths

val dotenv = Dotenv.load()
val dataDir = dotenv.get("DATA_DIR").let { Paths.get(it).toFile() }.also { it.mkdirs() }
dataDir

/home/j.zeidler@GDAAG.DE/private/ba3/bachelor-thesis/data

In [3]:
import org.bson.BsonDocument
import org.litote.kmongo.*

val client = KMongo.createClient("mongodb://localhost:42692/")
val db = client.getDatabase("s5_snyk_libio")
val libioVuln = db.getCollection<BsonDocument>("libioVuln")

In [4]:
import com.mongodb.client.*

private class MongoCursorIterable<T>(private val cursor: MongoCursor<T>) : MongoCursor<T> by cursor, Iterable<T> {

    override fun iterator(): Iterator<T> = cursor
}

private fun <T> MongoIterable<T>.kCursor(): MongoCursorIterable<T> = MongoCursorIterable(iterator())

fun <T, R> MongoIterable<T>.useCursor(block: (Iterable<T>) -> R): R {
    return kCursor().use(block)
}

In [5]:
val vulnGavs = libioVuln.find().useCursor { blk ->
    blk.map {
        it["vuln_gav"]!!.asString().value
    }
}.toSet()

vulnGavs.count()

3853

In [6]:
val jarsDir = dataDir.resolve("interim/jars").also { it.mkdirs() }

fun gavToJarUrl(packageGAV: String): String
{
    val parts = packageGAV.split(":")
    return parts[0].replace('.', '/') + "/" + parts[1] + "/" + parts[2] + "/" + parts[1] + "-" + parts[2] + ".jar"
}

In [7]:
val gavToJar = vulnGavs
    .map {
        val jarPath = jarsDir.resolve(gavToJarUrl(it))
        if (!jarPath.isFile()) null
        else it to jarPath
    }
    .filterNotNull()
    .toMap()
gavToJar.count()  // 3856

3852

In [8]:
import java.io.FileNotFoundException
import java.util.zip.ZipFile

fun classExistsInJar(jar: File, classFilePath: String): Boolean {
    if (!jar.isFile()) {
        throw FileNotFoundException("jar file not found: ${jar.absolutePath}")
    }
    
    val jarAsZip = ZipFile(jar)
    val entry = jarAsZip.getEntry(classFilePath)
    if (entry == null)
        return false
    
    return true
}

In [9]:
import org.bson.BsonArray

val filteredItems = libioVuln.find().useCursor { blk ->
    blk.filter {
        gavToJar.containsKey(it["vuln_gav"]!!.asString().value)
    }.map {
        val jar = gavToJar[it["vuln_gav"]!!.asString().value]!!
        val modifiedClassesInJar = it["vuln_classes"]!!.asArray().values.filter {
            classExistsInJar(
                jar, it.asString().value.replace('.', '/') + ".class"
            )
        }

        if (modifiedClassesInJar.isEmpty()) return@map null

        it["vuln_classes"] = BsonArray(modifiedClassesInJar)
        it
    }.filterNotNull()
}

In [10]:
filteredItems.count()

6864

In [11]:
filteredItems.map { it["vuln_gav"]!!.asString().value }.toSet().count()

3290

In [12]:
libioVuln.drop()
libioVuln.insertMany(filteredItems)

AcknowledgedInsertManyResult{insertedIds={0=BsonObjectId{value=67fd163c92aface2d37dce9e}, 1=BsonObjectId{value=67fd163c92aface2d37dce9f}, 2=BsonObjectId{value=67fd163c92aface2d37dcea0}, 3=BsonObjectId{value=67fd163c92aface2d37dcea1}, 4=BsonObjectId{value=67fd163c92aface2d37dcea2}, 5=BsonObjectId{value=67fd163c92aface2d37dcea3}, 6=BsonObjectId{value=67fd163c92aface2d37dcea4}, 7=BsonObjectId{value=67fd163c92aface2d37dcea5}, 8=BsonObjectId{value=67fd163c92aface2d37dcea6}, 9=BsonObjectId{value=67fd163c92aface2d37dcea7}, 10=BsonObjectId{value=67fd163c92aface2d37dcea8}, 11=BsonObjectId{value=67fd163c92aface2d37dcea9}, 12=BsonObjectId{value=67fd163c92aface2d37dceaa}, 13=BsonObjectId{value=67fd163c92aface2d37dceab}, 14=BsonObjectId{value=67fd163c92aface2d37dceac}, 15=BsonObjectId{value=67fd163c92aface2d37dcead}, 16=BsonObjectId{value=67fd163c92aface2d37dceae}, 17=BsonObjectId{value=67fd163c92aface2d37dceaf}, 18=BsonObjectId{value=67fd163c92aface2d37dceb0}, 19=BsonObjectId{value=67fd163c92aface